In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import pandas as pd

2024-11-10 16:19:54.753456: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 16:19:54.767409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731248394.785252   13244 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731248394.791268   13244 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 16:19:54.808802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Load the dataset
file_path = '/home/moegho/Desktop/490_Project/Data_processing/clean_labeled_news_dataset.csv'
df = pd.read_csv(file_path)

# Step 1: Check for non-string entries in the 'text' column and remove them
non_string_entries = df[~df['text'].apply(lambda x: isinstance(x, str))]
if not non_string_entries.empty:
    print("Non-string entries found in 'text' column:")
    print(non_string_entries)
    # Remove non-string entries from 'text' column
    df = df[df['text'].apply(lambda x: isinstance(x, str))].reset_index(drop=True)

# Step 2: Reindex the 'label' column to match the filtered 'text' column
# This step ensures that the label column only includes rows with valid text entries
df['label'] = df['label'].iloc[df.index].reset_index(drop=True)

# Final consistency check between 'text' and 'label' columns
if len(df['text']) != len(df['label']):
    raise ValueError("Inconsistent lengths after filtering. Please check the data.")
else:
    print("Data is ready for tokenization: 'text' and 'label' columns are consistent in length.")


Non-string entries found in 'text' column:
      text  label
294    NaN      9
11581  NaN      8
Data is ready for tokenization: 'text' and 'label' columns are consistent in length.


In [3]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the 'text' column
encodings = tokenizer(
    df['text'].tolist(), 
    padding=True, 
    truncation=True, 
    max_length=128, 
    return_tensors="pt"
)

# Prepare the labels as tensors
labels = torch.tensor(df['label'].tolist())

In [4]:
# Save the tokenized data and labels
torch.save(encodings['input_ids'], 'input_ids.pt')
torch.save(encodings['attention_mask'], 'attention_mask.pt')
torch.save(labels, 'labels.pt')

In [5]:
# input_ids = torch.load('input_ids.pt')
# attention_mask = torch.load('attention_mask.pt')
# labels = torch.load('labels.pt')

# for loading